In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
import os,cv2

In [6]:
import os
import cv2
import numpy as np

directory = "dataset/faces/"
X = []
y = []
target = []
person_id = 0
h = w = 300
n_samples = 0
classes = []

for person_name in os.listdir(directory):
    dir_path = os.path.join(directory, person_name)
    if os.path.isdir(dir_path):  # Check if it is a directory
        classes.append(person_name)
        for image_name in os.listdir(dir_path):
            image_path = os.path.join(dir_path, image_name)
            image = cv2.imread(image_path)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            resized_image = cv2.resize(gray, (h, w))
            v = resized_image.flatten()
            X.append(v)
            n_samples += 1
            y.append(person_id)
            target.append(person_name)
        person_id += 1

X = np.array(X)
y = np.array(y)
target_names = np.array(target)
n_features = X.shape[1]

print(f"Number of samples: {n_samples}")
print(f"Number of features: {n_features}")
print(f"Classes: {classes}")


Number of samples: 450
Number of features: 90000
Classes: ['Amitabh', 'Alia', 'Ileana', 'Aamir', 'Akshay', 'Deepika', 'Farhan', 'Disha', 'Ajay']


In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)
n_components=75
pca=PCA(n_components=n_components,svd_solver='randomized',whiten=True)
pca.fit(X_train)
eigenfaces=pca.components_.reshape((n_components,h,w))
#eigenface_titles=["eigenface %d" % i for i in range(eigenfaces.shape[0])]
#plot_gallery(eigenfaces,eigenface_titles,h,w)
#plt.show
X_train_pca=pca.transform(X_train)
X_test_pca=pca.transform(X_test)
lda=LinearDiscriminantAnalysis()
lda.fit(X_train_pca,y_train)
X_train_lda=lda.transform(X_train_pca)
X_test_lda=lda.transform(X_test_pca)

In [32]:
clf=MLPClassifier(random_state=42,hidden_layer_sizes=(40,40),max_iter=100,verbose=True)
clf.fit(X_train_lda,y_train)


NameError: name 'MLPClassifier' is not defined

In [10]:
y_predictions = []
y_real = []

# Collect predictions and true labels
for test_faces in X_test_lda:
    # Get the class probabilities for the test face
    prob = clf.predict_proba([test_faces])[0]
    # Find the class ID with the highest probability
    class_id = np.argmax(prob)
    # Append the predicted class ID to y_predictions
    y_predictions.append(class_id)
    # Append the true class ID to y_real
    y_real.append(y_test[np.where(X_test_lda == test_faces)[0][0]])

# Convert lists to numpy arrays
y_predictions = np.array(y_predictions)
y_real = np.array(y_real)

# Initialize variables for accuracy calculation and storing results
prediction_title = []
true_positive = 0

# Iterate through the predictions and true labels
for i in range(y_predictions.shape[0]):
    true_name = classes[y_real[i]]
    pred_name = classes[y_predictions[i]]
    result = 'pred: %s, pr: %s \ntrue: %s' % (pred_name, str(y_real[i]), true_name)
    prediction_title.append(result)
    # Check if the prediction is correct
    if true_name == pred_name:
        true_positive += 1

# Calculate accuracy
accuracy = true_positive * 100 / y_predictions.shape[0]
print('Accuracy:', accuracy)


Accuracy: 70.0


In [12]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical


In [ ]:

# Load and preprocess the dataset
directory = "dataset/faces/"
X = []
y = []
target = []
person_id = 0
h, w = 300, 300
n_samples = 0
classes = []

for person_name in os.listdir(directory):
    dir_path = os.path.join(directory, person_name)
    if os.path.isdir(dir_path):  # Check if it is a directory
        classes.append(person_name)
        for image_name in os.listdir(dir_path):
            image_path = os.path.join(dir_path, image_name)
            image = cv2.imread(image_path)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            resized_image = cv2.resize(gray, (h, w))
            v = resized_image.flatten()
            X.append(v)
            n_samples += 1
            y.append(person_id)
            target.append(person_name)
        person_id += 1

X = np.array(X)
y = np.array(y)
target_names = np.array(target)
n_features = X.shape[1]

print(f"Number of samples: {n_samples}")
print(f"Number of features: {n_features}")
print(f"Classes: {classes}")

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [ ]:
# Apply PCA
n_components = 75
pca = PCA(n_components=n_components, svd_solver='auto', whiten=True)
pca.fit(X_train)
eigenfaces = pca.components_.reshape((n_components, h, w))
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# Apply LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_pca, y_train)
X_train_lda = lda.transform(X_train_pca)
X_test_lda = lda.transform(X_test_pca)

In [ ]:

# Convert y_train and y_test to one-hot encoded format
y_train = to_categorical(y_train, num_classes=len(classes))
y_test = to_categorical(y_test, num_classes=len(classes))

In [92]:


# Define the model
model = Sequential([
    Dense(1024, activation='relu', input_shape=(X_train_lda.shape[-1],)),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(len(classes), activation='softmax')  # Number of classes for the output
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_lda, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
accuracy = model.evaluate(X_test_lda, y_test)[1] * 100
print(f'Accuracy: {accuracy}')

# Predictions and accuracy calculation
y_predictions = model.predict(X_test_lda)
y_predictions = np.argmax(y_predictions, axis=1)
y_real = np.argmax(y_test, axis=1)

prediction_title = []
true_positive = 0

for i in range(y_predictions.shape[0]):
    true_name = classes[y_real[i]]
    pred_name = classes[y_predictions[i]]
    result = 'pred: %s, pr: %s \ntrue: %s' % (pred_name, str(y_real[i]), true_name)
    prediction_title.append(result)
    # Check if the prediction is correct
    if true_name == pred_name:
        true_positive += 1

# Calculate accuracy
accuracy = true_positive * 100 / y_predictions.shape[0]
# Definaccuracy = true_positive * 100 / y_predictions.shape[0]
print('Accuracy:', accuracy)


Number of samples: 450
Number of features: 90000
Classes: ['Amitabh', 'Alia', 'Ileana', 'Aamir', 'Akshay', 'Deepika', 'Farhan', 'Disha', 'Ajay']
Epoch 1/10
9/9 [==============================] - 0s 9ms/step - loss: 1.6817 - accuracy: 0.4965 - val_loss: 0.9984 - val_accuracy: 0.7917
Epoch 2/10
9/9 [==============================] - 0s 3ms/step - loss: 0.5563 - accuracy: 0.8646 - val_loss: 0.6780 - val_accuracy: 0.7917
Epoch 3/10
9/9 [==============================] - 0s 3ms/step - loss: 0.3954 - accuracy: 0.8542 - val_loss: 0.7200 - val_accuracy: 0.8194
Epoch 4/10
9/9 [==============================] - 0s 3ms/step - loss: 0.2822 - accuracy: 0.8993 - val_loss: 0.9536 - val_accuracy: 0.8056
Epoch 5/10
9/9 [==============================] - 0s 3ms/step - loss: 0.2363 - accuracy: 0.9167 - val_loss: 0.7641 - val_accuracy: 0.7778
Epoch 6/10
9/9 [==============================] - 0s 3ms/step - loss: 0.1633 - accuracy: 0.9583 - val_loss: 0.6464 - val_accuracy: 0.8194
Epoch 7/10
9/9 [===========